# **Inference Proyek Fundamental Deep Learning**
- **Nama:** Muhammad Makarim
- **Email:** mmakarim9@student.ub.ac.id
- **ID Dicoding:** MC006D5Y1427

## **Import Semua Packages/Library yang Digunakan**

In [127]:
# Import Library untuk Inference
import pickle
import numpy as np
import tensorflow as tf
import re
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import _VectorizerMixin

# Nonaktifkan peringatan warning
warnings.filterwarnings('ignore')

# Konfigurasi TensorFlow (opsional)
tf.get_logger().setLevel('ERROR')

## **Load Model untuk Inference**

In [105]:
# Load Data
ulasan = pd.read_csv('data_bersih_berlabel.csv')

In [106]:
# Ekstrak Data untuk Klasifikasi
texts = ulasan['stopword_removal'].tolist()
labels = ulasan['sentiment_label'].tolist()

### **Load Model**

In [211]:
# Memuat Vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Memuat SVM Model
with open('svm_tfidf_8020_model.pkl', 'rb') as f:
    svm_tfidf_8020 = pickle.load(f)

with open('svm_tfidf_6040_model.pkl', 'rb') as f:
    svm_tfidf_6040 = pickle.load(f)

with open('svm_ngram_8020_model.pkl', 'rb') as f:
    svm_ngram_8020 = pickle.load(f)

with open('svm_bow_8020_model.pkl', 'rb') as f:
    svm_bow_8020 = pickle.load(f)

# Memuat Random Forest Model
with open('rf_tfidf_8020_model.pkl', 'rb') as f:
    rf_tfidf_8020 = pickle.load(f)

with open('rf_tfidf_7030_model.pkl', 'rb') as f:
    rf_tfidf_7030 = pickle.load(f)

# Memuat LSTM Model
lstm_tfidf_8020 = tf.keras.models.load_model('lstm_tfidf_8020_model.h5')

## **Preprocessing**

### **Membuat Data Baru Untuk Prediksi**

In [193]:
#Data Baru
new_texts = [
    "suka banget ini aplikasi bagus!",  # sentimen positif
    "tidak bisa masuk gagal terus",  # sentimen negatif
    "kualitasnya biasa saja."  # netral
]

### **Preprocessing Teks**

In [194]:
# Preprocessing Teks SVM dan RF
new_texts_cleaned = []
for text in new_texts:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join(text.split())
    new_texts_cleaned.append(text)

In [195]:
# Preprocessing untuk Model LSTM:
new_sequences = tokenizer.texts_to_sequences(new_texts_cleaned)
new_padded = pad_sequences(new_sequences, maxlen=100)

In [196]:
# Transform data baru dengan TF-IDF
data_new_tfidf = tfidf_vectorizer.transform(new_texts_cleaned)

In [197]:
ngram_vectorizer_new = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=2000,
)

ngram_vectorizer_new.fit(texts)

TfidfVectorizer(max_features=2000, ngram_range=(1, 2))

In [198]:
# Transform data baru dengan N-Gram
data_new_ngram = ngram_vectorizer_new.transform(new_texts_cleaned)
data_new_ngram_dense = data_new_ngram.toarray()

In [199]:
bow_vectorizer_new = TfidfVectorizer(
    max_features=8618,
)

bow_vectorizer_new.fit(texts)

TfidfVectorizer(max_features=8618)

In [200]:
data_new_bow = bow_vectorizer_new.transform(new_texts_cleaned)
data_new_bow_dense = data_new_bow.toarray()

## **Prediksi**

### ***N-Gram dan BoW***

In [201]:
svm_ngram_8020_pred = svm_ngram_8020.predict(data_new_ngram_dense)

In [202]:
svm_bow_8020_pred = svm_bow_8020.predict(data_new_bow_dense)

In [203]:
print(svm_ngram_8020.n_features_in_)

2000


In [204]:
print(data_new_tfidf.shape[1])

10000


### **TF-IDF**

In [205]:
# SVM Prediksi
svm_tfidf_8020_pred = svm_tfidf_8020.predict(data_new_tfidf)
svm_tfidf_6040_pred = svm_tfidf_6040.predict(data_new_tfidf)

In [206]:
# Random Forest Prediksi
rf_tfidf_8020_pred = rf_tfidf_8020.predict(data_new_tfidf)
rf_tfidf_7030_pred = rf_tfidf_7030.predict(data_new_tfidf)

In [207]:
# LSTM Prediksi
lstm_preds_classes = np.argmax(lstm_tfidf_8020.predict(new_padded), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


## **Hasil**

In [208]:
# Mapping label numerik ke label string
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Tampilkan hasil prediksi
print("\nSentiment Prediction Results:")
for i, text in enumerate(new_texts):
    print(f"Text: {text}")
    print(f"LSTM Prediction: {label_map[lstm_preds_classes[i]]}")
    print(f"SVM N-Gram Prediction: {svm_ngram_8020_pred[i]}")
    print(f"SVM BoW Prediction: {svm_bow_8020_pred[i]}")
    print(f"SVM TF-IDF (8020) Prediction: {label_map[svm_tfidf_8020_pred[i]]}")
    print(f"SVM TF-IDF (6040) Prediction: {label_map[svm_tfidf_6040_pred[i]]}")
    print(f"RF TF-IDF (8020) Prediction: {label_map[rf_tfidf_8020_pred[i]]}")
    print(f"RF TF-IDF (7030) Prediction: {label_map[rf_tfidf_7030_pred[i]]}")
    print("-" * 50)


Sentiment Prediction Results:
Text: suka banget ini aplikasi bagus!
LSTM Prediction: neutral
SVM N-Gram Prediction: negatif
SVM BoW Prediction: netral
SVM TF-IDF (8020) Prediction: positive
SVM TF-IDF (6040) Prediction: positive
RF TF-IDF (8020) Prediction: positive
RF TF-IDF (7030) Prediction: positive
--------------------------------------------------
Text: tidak bisa masuk gagal terus
LSTM Prediction: neutral
SVM N-Gram Prediction: netral
SVM BoW Prediction: netral
SVM TF-IDF (8020) Prediction: negative
SVM TF-IDF (6040) Prediction: negative
RF TF-IDF (8020) Prediction: negative
RF TF-IDF (7030) Prediction: negative
--------------------------------------------------
Text: kualitasnya biasa saja.
LSTM Prediction: neutral
SVM N-Gram Prediction: netral
SVM BoW Prediction: netral
SVM TF-IDF (8020) Prediction: neutral
SVM TF-IDF (6040) Prediction: neutral
RF TF-IDF (8020) Prediction: neutral
RF TF-IDF (7030) Prediction: neutral
--------------------------------------------------
